<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing,-exploration-and-export-of-app-reviews" data-toc-modified-id="Preprocessing,-exploration-and-export-of-app-reviews-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing, exploration and export of app reviews</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Ratings" data-toc-modified-id="Ratings-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ratings</a></span></li><li><span><a href="#Detect-language" data-toc-modified-id="Detect-language-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Detect language</a></span></li><li><span><a href="#Sort-data" data-toc-modified-id="Sort-data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Sort data</a></span></li><li><span><a href="#Export-data" data-toc-modified-id="Export-data-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Export data</a></span></li></ul></li></ul></div>

# Preprocessing, exploration and export of app reviews

We have scraped reviews on a [specific app](https://apps.apple.com/fr/app/airvisual-qualit%C3%A9-de-lair/id1048912974#see-all/reviews) in the French appstore. This app is related to air quality. Our goal is to analyse these reviews to try to find out about :
* usages
* most relevant app features
* "missing" app features, or features that users would like the app to have
* technical issues.

Data preparation will be key to help analyse the reviews, such as sorting reviews according to selected criteria.  This will also give us the opportunity to test some NLP tools as needed (language detection, sentiment analysis...).

In [1]:
import pandas as pd
from langdetect import detect
import warnings
warnings.filterwarnings('ignore')

## Load data

In [2]:
#filename = 'app_reviews_airvisual-air-quality-forecast_1048912974.json'
filename ='app_reviews_en.json' # from Great Britain appstore

In [3]:
df = pd.read_json(filename)

In [4]:
df.head()

,review_id,rating,title,review_date,user_name,review,response_id,dev_response,response_date
0,4915894551,5,A tragic reality,2019-10-10T01:06:52Z,Si & Ro,"Amazingly helpful app, both for health and as ...",NaN,NaN,NaN
1,5387399470,5,Really useful information,2020-01-11T01:19:08Z,Suez62,We have an ‘eco’ woodburner and as an asthmati...,NaN,NaN,NaN
2,4057061182,5,Air quality app,2019-04-25T07:07:47Z,# alone at xmas,I was introduced to this app via a friend who ...,NaN,NaN,NaN
3,4858351034,5,Accurate & Reliable,2019-09-29T07:38:33Z,r2thebizel,"Very clear, easy to use and engaging. Very rel...",NaN,NaN,NaN
4,3883883779,5,Best aqi app yet,2019-03-15T12:07:55Z,jhugs43,"I’ve tried several aqi apps over the years, mo...",NaN,NaN,NaN


In [5]:
df.columns

Index(['review_id', 'rating', 'title', 'review_date', 'user_name', 'review',
       'response_id', 'dev_response', 'response_date'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   review_id      243 non-null    int64  
 1   rating         243 non-null    int64  
 2   title          243 non-null    object 
 3   review_date    243 non-null    object 
 4   user_name      243 non-null    object 
 5   review         243 non-null    object 
 6   response_id    2 non-null      float64
 7   dev_response   2 non-null      object 
 8   response_date  2 non-null      object 
dtypes: float64(1), int64(2), object(6)
memory usage: 17.2+ KB


There are 334 reviews. There are only 3 responses from the developer.

In order to verify that it was not due to an issue with web scraping, we can go back to the [see-all reviews page](https://apps.apple.com/fr/app/airvisual-qualit%C3%A9-de-lair/id1048912974#see-all/reviews). After loading all the reviews, we could see only 3 responses from the developer (as of 26/06/2020).

In [ ]:
df['review']

## Ratings

In [ ]:
# assess the distribution of ratings
df['rating'].value_counts()

In [ ]:
# assess mean rating
df['rating'].mean()

## Detect language

In [ ]:
# Define a function to identify language and catch exceptions
def lang_detect(text):
    # use deterministic approach for language detection
    from langdetect import DetectorFactory
    DetectorFactory.seed = 0
    try:
        return detect(text)
    except:
        return "language not detected"

In [ ]:
# Detect the language used in the reviews
df['lang-r'] = df['review'].apply(lang_detect)

In [ ]:
# What are the detected languages?
df['lang-r'].unique()

In [ ]:
# What is the distribution of the detected languages?
df['lang-r'].value_counts()

As expected, most reviews are detected as being in French, since reviews were collected from the French appstore. However 70 reviews are detected as being in English.

In [ ]:
# Look at reviews where the language could not be detected
df.loc[df['lang-r']=='language not detected']

Most of reviews where the language could not be detected are made of emoticones or mathematical symbols. Detecting the language using the title should help. However, we're focusing on reviews where enough words to identify information we're looking for. Then, we'll discard the reviews where the language could not be detected.

In [ ]:
# Look at the reviews where the languages is neither French, English or not detected
df.loc[(df['lang-r']!='fr')
       &(df['lang-r']!='en')
       &(df['lang-r']!='language not detected')].head(27)

When the review is short, especially if there is a typo, the language detection is not correct. Let's check if language could be detected using the title.

In [ ]:
df['lang-t'] = df.loc[(df['lang-r']!='fr')
       &(df['lang-r']!='en')
       &(df['lang-r']!='language not detected')]['title'].apply(lang_detect)

In [ ]:
df_2 = df.loc[(df['lang-r']!='fr')
       &(df['lang-r']!='en')
       &(df['lang-r']!='language not detected')][['title', 'review', 'lang-r', 'lang-t']]

In [ ]:
df_2.head()

In [ ]:
df_2['lang-t'].value_counts()

In [ ]:
df_2.sort_values(by='lang-t', inplace=True)

In [ ]:
df_2.head(27)

In the final dataframe, we'll keep only the reviews where French or English languages have been detected either in the review or in the title.

In [ ]:
len(df.loc[(df['lang-r']=='fr')|(df['lang-r']=='en')
          |(df['lang-t']=='fr')|(df['lang-t']=='en')])

In [ ]:
dfout = df.loc[(df['lang-r']=='fr')|(df['lang-r']=='en')
          |(df['lang-t']=='fr')|(df['lang-t']=='en')]

In [ ]:
# we define a new column 'lang': it's the language of the review if it's 
# in French or in English
dfout.loc[(df['lang-r']=='fr')|(df['lang-r']=='en'),'lang'] = df.loc[(df['lang-r']=='fr')|(df['lang-r']=='en'),'lang-r']

In [ ]:
# check the data
dfout.head()

In [ ]:
# check cases where language review is not French or English
dfout.loc[(df['lang-r']!='fr')&(df['lang-r']!='en')]

In [ ]:
# else, the value for 'lang' is the language detected in the title (French or English)
dfout.loc[(df['lang-r']!='fr')&(df['lang-r']!='en'),'lang'] = dfout.loc[(df['lang-r']!='fr')&(df['lang-r']!='en'),'lang-t'] 

In [ ]:
# check again cases where language review is not French or English
dfout.loc[(df['lang-r']!='fr')&(df['lang-r']!='en')]

## Sort data

In [ ]:
dfout.sort_values(by=['lang','rating','review_date'], inplace = True, ascending = False)

In [ ]:
dfout

## Export data

In [ ]:
# export to csv
dfout.to_csv('app_reviews_airvisual-air-quality-forecast_1048912974_by_lang.csv', encoding='utf-8-sig', sep =';')